In [1]:
import pandas as pd
import numpy as np
import calendar
import os

from datetime import datetime
from elasticsearch import Elasticsearch,helpers
from elasticsearch.helpers import parallel_bulk
from elasticsearch.helpers import streaming_bulk
from es_pandas import es_pandas

# 导入datetime库
import datetime
import random
#import modin.pandas as mpd

#import commond.ipynb from same folder
import import_ipynb
from common import *

importing Jupyter notebook from common.ipynb
03 Aug 2020 DAQ FilesPQC887


In [2]:
# generate nums of rows in dataframe
def random_dataframe(nums):
    #using numpy's randint
   df = pd.DataFrame(np.random.randint(0,nums,size=(15, 41)), columns=list(get_interests_columns()))
   return df

ds = random_dataframe(100)
ds

,Date,Time,Comm Status,CraneNumber,Weight,Frequency,Spreader_20FT,Spreader_40FT,Spreader_45FT,Spreader_Twin,...,OT_Control STS_Gantry Right (Controller),OT_Control STS_Gantry Left (Controller),OT_Control STS_Boom Up (Controller),OT_Control STS_Boom Down (Controller),Trolley Position,Load Cell1 Data,Load Cell2 Data,Load Cell3 Data,Load Cell4 Data,SP STS_Landed
0,43,9,95,97,14,47,70,0,59,80,...,30,46,88,67,70,41,32,58,95,98
1,37,9,90,51,4,63,61,78,92,17,...,4,58,22,61,43,52,27,22,69,92
2,99,77,97,79,17,6,90,17,5,23,...,77,92,20,34,25,41,4,92,12,57
3,37,19,17,15,43,40,9,81,85,73,...,3,40,38,80,59,40,13,88,76,48
4,1,54,42,8,74,96,32,49,72,36,...,18,97,44,8,19,85,71,44,16,90
5,91,65,85,65,46,4,85,37,83,28,...,92,18,11,69,30,48,95,68,58,96
6,62,0,13,53,74,31,78,45,91,52,...,7,97,93,75,55,29,60,53,88,50
7,81,9,73,87,82,41,97,58,75,21,...,62,68,80,63,65,56,84,83,8,98
8,26,23,6,5,88,78,6,12,55,79,...,59,31,91,74,28,8,42,71,97,58
9,37,23,37,16,96,68,69,0,61,82,...,36,83,73,16,93,23,75,18,98,30


In [3]:
def callback_ingestion(data_set):
    # Information of es cluseter
    es_host = 'localhost:9200'
    index = 'compact_daq_index'

    # crete es_pandas instance
    ep = es_pandas(es_host)
    # init template if you want
    doc_type = 'records'

    ep.init_es_tmpl(data_set, doc_type)

    # Example of write data to es, use the template you create
    ep.to_es(data_set, index, doc_type=doc_type, thread_count=4, chunk_size=7200)


In [4]:
def print_time(msg):
    # datetime object containing current date and time
    now = datetime.datetime.now()
    # dd/mm/YY H:M:S
    dt_string = now.strftime("%Y-%m-%d %H:%M:%S.%f%Z")
    print(msg+dt_string)

In [5]:
# return dataframe by timerange
def ingest_bulk_dataframe(start_date, end_date, crane_number, callback=None):
    # 获取时间间隔
    delta = end_date - start_date
    
    ds = pd.DataFrame()
    const_ds = pd.DataFrame()

    const_ds.at[0, "CraneNumber"] = crane_number
    const_ds.at[1, "CraneNumber"] = crane_number
    const_ds.at[0, "Twistlock"] = 0
    const_ds.at[1, "Twistlock"] = 1
    const_ds.at[0, "Comm Status"] = 'OK'
    const_ds.at[1, "Comm Status"] = 'False'
    const_ds.at[0, "Asset Type"] = 'QC'
    const_ds.at[1, "Asset Type"] = 'YC'
    
    col_list = get_interests_columns_simple()
    # 遍历获得每一个时间点
    for i in range(int(delta.total_seconds())+1):
        #print_time("randint")
        ss = random.randint(10, 100)
        temp = start_date + datetime.timedelta(seconds=i)
        dt1 = temp + datetime.timedelta(milliseconds=ss)
        dt2 = dt1 + datetime.timedelta(milliseconds=500)
        
        #print_time("start create df ")
        df = pd.DataFrame(np.random.randint(10,1000,size=(2, len(col_list))), columns=list(col_list))
        #print_time("finish create df ")
        
        df.at[0, "timestamp"] = dt1.astimezone().isoformat()
        df.at[1, "timestamp"] = dt2.astimezone().isoformat()

        #print_time("finish add df ")
        new_df = pd.concat([df, const_ds], axis=1)
        del df
        #print_time("finish cancat df ")
        ds = pd.concat([ds, new_df], axis=0)
        #ds = ds.append(new_df)
        #print(ds)
        #print_time("finish append df ")
        
        if ds.shape[0] >= 14400:
           #print_time("start ingestion to ES ")
           callback(ds)
           print_time("finish ingestion to ES ")
           msg = dt2.strftime("%d-%b-%Y (%H:%M:%S.%f)")
           print('Ingest %s  is completed!' % msg)
           ds = ds[0:0]
           
    if ds.shape[0] > 0:
       callback(ds) 


In [ ]:
# ingest whole year data for all crane
for i in range(100):
    crane = 'PQC' + str(random.randint(800, 900))
    # 创建开始时间点和结束时间点
    dt_start = datetime.datetime(2019,7,1,0,0,1)
    dt_end =   datetime.datetime(2020,7,1,0,0,0)
    ingest_bulk_dataframe(dt_start, dt_end, crane, callback_ingestion)
    print('Ingest %s  is completed!' % crane)

 92% (13307 of 14400) |################  | Elapsed Time: 0:00:00 ETA:   0:00:00/home/searoc/anaconda3/lib/python3.7/site-packages/elasticsearch/connection/base.py:177: ElasticsearchDeprecationWarning: [types removal] Specifying types in bulk requests is deprecated.
  warnings.warn(message, category=ElasticsearchDeprecationWarning)


finish ingestion to ES 2020-07-12 21:58:34.213842
Ingest 01-Jul-2019 (02:00:00.588000)  is completed!


 94% (13671 of 14400) |################# | Elapsed Time: 0:00:00 ETA:   0:00:00

finish ingestion to ES 2020-07-12 21:59:03.569268
Ingest 01-Jul-2019 (04:00:00.573000)  is completed!


 97% (14036 of 14400) |################# | Elapsed Time: 0:00:00 ETA:   0:00:00

...

finish ingestion to ES 2020-07-15 13:40:42.782354
Ingest 09-Apr-2020 (08:00:00.585000)  is completed!


 92% (13307 of 14400) |################  | Elapsed Time: 0:00:00 ETA:   0:00:00

finish ingestion to ES 2020-07-15 13:41:12.559126
Ingest 09-Apr-2020 (10:00:00.519000)  is completed!


 92% (13307 of 14400) |################  | Elapsed Time: 0:00:00 ETA:   0:00:00

finish ingestion to ES 2020-07-15 13:41:48.387498
Ingest 09-Apr-2020 (12:00:00.587000)  is completed!


In [53]:
dt_start = datetime.datetime(2017,1,1,0,0,1)
dt_end =   datetime.datetime(2017,12,31,0,4,2)

delta = dt_end - dt_start
print_time(str(delta.days*24*60*60+delta.seconds+1))
print_time(str(delta.total_seconds()))


31449842 04/07/2020 22:00:51.846222
31449841.0 04/07/2020 22:00:51.846875


In [7]:
def get_day_time_range(start_day, end_day):
    # 获取时间间隔
    delta = start_day - start_day
    ds = pd.DataFrame()
    
    for i in range(delta.days+1):
        df.at[0, "Start"] = start_day.date